Installations

In [ ]:
!pip install -U langchain-community
!pip install agno
!pip install google-search-results
!pip install chromadb
!pip install groq

Getting knowlegebase from kaggle


In [ ]:
import kagglehub
path = kagglehub.dataset_download("thedevastator/dataset-for-solving-math-word-problems")



In [ ]:
import os
print("Path to dataset files:", path)
os.path.exists(path)
data= os.path.join(path,"train.csv")
import pandas as pd
from langchain.docstore.document import Document
df = pd.read_csv(data)

df.columns

Path to dataset files: /kaggle/input/dataset-for-solving-math-word-problems


Index(['Problem', 'Rationale', 'options', 'correct', 'annotated_formula',
       'linear_formula', 'category'],
      dtype='object')

Processing the data for vector databse

In [ ]:
docs = []
for _, entry in df.iterrows():
    content = f"""Question: {entry['Problem']} {entry['options']}
Answer: {entry['Rationale']} | Correct: {entry['correct']}
Formulae: {entry['annotated_formula']},{entry['linear_formula']}
Category: {entry['category']}"""

    doc = Document(page_content=content, metadata={"answer": entry["correct"]})
    docs.append(doc)

# Optional: limit for testing
docs = docs[:2000]

embedding the data and making the vectordb


In [ ]:
from uuid import uuid4

from langchain_core.documents import Document
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from uuid import uuid4
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=hf,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)


uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)

In [ ]:
print(vector_store._collection.count())

2000


Setting up agents for efficient swith between database and web search also integrating a Streamlit UI



In [ ]:
%%writefile app.py
import streamlit as st
import json
from datetime import datetime
import warnings

from agno.models.groq import Groq
from agno.agent import Agent
from agno.tools.serpapi import SerpApiTools
from agno.embedder.huggingface import HuggingfaceCustomEmbedder
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.chroma import ChromaDb

warnings.filterwarnings("ignore", category=ResourceWarning)

MODEL_ID = "llama-3.3-70b-versatile"
FEEDBACK_LOG = "feedback_log.jsonl"

my_model = Groq(id=MODEL_ID)

classifier_agent = Agent(
    name="Classifier Agent",
    role="Classify questions",
    model=my_model,
    instructions="Return only 'Math' or 'Not Math' based on the question.",
    show_tool_calls=False,
    markdown=False,
)

def classify_question_type(question: str) -> str:
    prompt = f"Classify the following question as either 'Math' or 'Not Math':\n\nQuestion: {question}\n\nAnswer:"
    result = classifier_agent.run(prompt)
    return result.content.strip()


web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=my_model,
    tools=[SerpApiTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)




knowledge_base = vector_store



knowledge_base.load(recreate=False)  #used for the loading the database will comment after first run

agent_team = Agent(
    team=[web_agent],
    model=my_model,
    knowledge=knowledge_base,
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)


st.set_page_config(page_title="Math Agent", layout="wide")
st.title("🧮 Your Math Professor")


if "response" not in st.session_state:
    st.session_state.response = None
if "question" not in st.session_state:
    st.session_state.question = ""
if "feedback" not in st.session_state:
    st.session_state.feedback = None


with st.form("question_form"):
    question = st.text_input("Enter a math question:", placeholder="e.g., What is the derivative of x^3 + 3")
    submit = st.form_submit_button("Ask Agent")


if submit and question:
    q_type = classify_question_type(question)
    if q_type.lower() != "math":
        st.error("❌ Only math-related questions are allowed. Try rephrasing.")
    else:
        with st.spinner("Thinking..."):
            response = agent_team.run(question)
            st.session_state.response = response.content
            st.session_state.question = question
            st.session_state.feedback = None


if st.session_state.response:
    st.markdown("### 🤖 Agent's Answer")
    st.markdown(st.session_state.response)

    st.markdown("### 🧠 Human Feedback")
    st.session_state.feedback = st.radio(
        "Was the answer helpful?",
        ["Yes", "No"],
        key="feedback_radio"
    )

    corrected_answer = ""
    if st.session_state.feedback == "No":
        corrected_answer = st.text_area(
            "Provide a corrected answer (optional):",
            key="corrected_answer"
        )

    if st.button("Submit Feedback"):
        log = {
            "timestamp": datetime.now().isoformat(),
            "question": st.session_state.question,
            "agent_answer": st.session_state.response,
            "feedback": st.session_state.feedback,
            "corrected_answer": corrected_answer if st.session_state.feedback == "No" else None
        }

        with open(FEEDBACK_LOG, "a") as f:
            f.write(json.dumps(log) + "\n")

        st.success("✅ Feedback logged")

        if corrected_answer:
            vector_store.add_documents([Document(page_content=f"Question: {st.session_state.question}, Answer: {corrected_answer}",metadata={"Answer":corrected_answer})],ids=[str(uuid4())])
            st.info("📚 Corrected answer added to knowledge base")


Overwriting app.py


In [ ]:
!curl https://loca.lt/mytunnelpassword

34.16.36.207

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://chubby-clocks-tie.loca.lt
^C
